# Fig2. Single cell dynamics of Diptericin activation

In [208]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import pickle
from scipy.stats import linregress
from matplotlib.colors import ListedColormap
from scipy.special import gamma
from scipy.optimize import minimize

In [18]:
%matplotlib qt
#%matplotlib inline

In [86]:
"""plot style"""
linewidth = 4
mpl.rc('axes', linewidth=linewidth)
mpl.rc('font', family='Arial')
fontsize = 24


colors = {'no_inj': [0.8, 0.8, 0.8],
         'mock': [0.4, 0.4, 0.4],
         'e.coli': [0, 0.4, 0]}

def style_axes(ax, fontsize=24):
    plt.minorticks_off()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.xaxis.set_tick_params(labelsize=20)
    ax.yaxis.set_tick_params(labelsize=20)
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    plt.tight_layout()
    
    return ax


## Plot individual traces from anterior and middle regions

In [129]:
df = pd.read_pickle(r'/media/brandon/Data2/Brandon/fly_immune/Lightsheet_Z1/2024_02_20_dpt-gfp_ca-Gal4_UAS-His-RFP_halocarbon_5x_timeseries/larva_2/manual_analysis/all_cells_quant.pkl')
df

,t,y,x,particle,mean_dpt
0,0.0,2395.963714,248.223467,101,478.072072
1,0.0,1444.137596,636.191829,41,280.274336
2,0.0,2569.999578,502.491128,94,376.898148
3,0.0,554.544004,595.305408,66,435.982609
4,0.0,1353.503119,584.670148,40,261.137931
...,...,...,...,...,...
9207,89.0,440.453056,588.304818,73,752.200000
9208,89.0,534.107136,553.581375,78,713.580357
9209,89.0,2617.257222,474.315887,99,507.008696
9210,89.0,1700.615819,528.341005,13,644.000000


In [130]:
"""group cells based on their ap position"""
df1 = df[df.y < 1000]
df2 = df[np.array(df.y.values >= 1000) * np.array(df.y.values < 2000)]
df3 = df[df.y > 2000]

In [172]:
"""plot individual traces colored by slope"""
slope_bins = np.linspace(0, 4, 100)
reds = np.linspace(1, 0, len(slope_bins))
greens = np.linspace(0, 1, len(slope_bins))
blues = np.ones(len(slope_bins))

t_fit_max = 40
t_fit_min = 10
minutes_per_time_point = 2

plt.figure()

# anterior
plt.subplot(121)
this_df = df1
nuc_ids = this_df.particle.unique()
slope_list_anterior = []
inten_6hpi_exp1_anterior = []
for i in range(len(nuc_ids)):
    sub_df = this_df[this_df.particle == nuc_ids[i]]
    sub_df = sub_df.sort_values(by='t')
    if any(sub_df.groupby(by='t').size().values > 1):
        continue
    t = sub_df.t
    dpt = sub_df.mean_dpt
    
    dpt = dpt[t <= t_fit_max]
    t = t[t <= t_fit_max]
    
    dpt = dpt[t > t_fit_min]
    t = t[t > t_fit_min]
    
    if len(t) > 4:
        slope, intercept, r, p, se = linregress(minutes_per_time_point * t, dpt)
        slope_list_anterior.append(slope)
        
        t6hpi_index = np.where(t * minutes_per_time_point / 60 + 5 > 6)[0][0]
        inten_6hpi_exp1_anterior.append(dpt.values[t6hpi_index])
        
        slope_index = np.where(np.abs(slope - slope_bins) == np.min(np.abs(slope - slope_bins)))[0][0]
        
        t = sub_df.t
        dpt = sub_df.mean_dpt
        plt.plot((t * 2 / 60) + 5, dpt.values, linewidth=2, color=[reds[slope_index], greens[slope_index], blues[slope_index]])

plt.xlabel('hours post infection', fontsize=fontsize)
plt.ylabel('mean diptericin-GFP \nintensity per cell (a.u.)', fontsize=fontsize)
plt.title('anterior', fontsize=fontsize)
plt.ylim([200, 1300])
plt.xlim([5, 8])
ax = style_axes(plt.gca())

# posterior
plt.subplot(122)
this_df = df2
nuc_ids = this_df.particle.unique()
slope_list_posterior = []
inten_6hpi_exp1_posterior = []
for i in range(len(nuc_ids)):
    sub_df = this_df[this_df.particle == nuc_ids[i]]
    sub_df = sub_df.sort_values(by='t')
    if any(sub_df.groupby(by='t').size().values > 1):
        continue
    t = sub_df.t
    dpt = sub_df.mean_dpt
    
    dpt = dpt[t <= t_fit_max]
    t = t[t <= t_fit_max]
    
    dpt = dpt[t > t_fit_min]
    t = t[t > t_fit_min]
    
    if len(t) > 4:
        slope, intercept, r, p, se = linregress(minutes_per_time_point * t, dpt)
        slope_list_posterior.append(slope)

        t6hpi_index = np.where(t * minutes_per_time_point / 60 + 5 > 6)[0][0]
        inten_6hpi_exp1_posterior.append(dpt.values[t6hpi_index])
        
        slope_index = np.where(np.abs(slope - slope_bins) == np.min(np.abs(slope - slope_bins)))[0][0]
        
        t = sub_df.t
        dpt = sub_df.mean_dpt
        plt.plot(t * minutes_per_time_point / 60 + 5, dpt.values, linewidth=2, color=[reds[slope_index], greens[slope_index], blues[slope_index]])

plt.xlabel('hours post infection', fontsize=fontsize)
plt.ylabel('mean diptericin-GFP \nintensity per cell (a.u.)', fontsize=fontsize)
plt.title('middle', fontsize=fontsize)
plt.ylim([200, 1300])
plt.xlim([5, 8])
ax = style_axes(plt.gca())


"""colorbar"""
N = len(slope_bins)
vals = np.ones((N, 4))
vals[:, 0] = reds
vals[:, 1] = greens
vals[:, 2] = blues
newcmp = ListedColormap(vals)
tmp_im = np.array([slope_bins, slope_bins])



[Text(1, 0, '0'), Text(1, 2, '2'), Text(1, 4, '4')]

In [173]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig2-SingleCellDynamics/anterior_vs_middle_traces.pdf')

In [174]:
"""plot a colorbar separately"""
fig, ax = plt.subplots()
cax = ax.imshow(tmp_im, cmap=newcmp)


# Add colorbar, make sure to specify tick locations to match desired ticklabels
cbar = fig.colorbar(cax, ticks=[0, 2, 4])
cbar.ax.set_yticklabels([0, 2, 4], fontsize=24)  # vertically oriented colorbar


[Text(1, 0, '0'), Text(1, 2, '2'), Text(1, 4, '4')]

In [175]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig2-SingleCellDynamics/traces_colorbar.pdf')

## Plot dot plot of median slope in anterior vs middle for two experiments 

In [132]:
"""assemble medians from experiment 2"""
df_exp2 = pd.read_pickle(r'/media/brandon/Data2/Brandon/fly_immune/Lightsheet_Z1/2024_02_14_dpt-gfp_ca-Gal4_UAS-His-RFP_halocarbon_5x_timeseries/larva_2/manual_analysis/all_cells_quant.pkl')
df_exp2_anterior = df_exp2[df_exp2.y < 1000]
df_exp2_middle = df[np.array(df.y.values >= 1000) * np.array(df.y.values < 2400)]


In [176]:
t_fit_max = 12
t_fit_min = 0
minutes_per_time_point = 5
start_hour = 6

# anterior
this_df = df_exp2_anterior
nuc_ids = this_df.particle.unique()
slope_list_exp2_anterior = []
inten_6hpi_exp2_anterior = []
for i in range(len(nuc_ids)):
    sub_df = this_df[this_df.particle == nuc_ids[i]]
    sub_df = sub_df.sort_values(by='t')
    if any(sub_df.groupby(by='t').size().values > 1):
        continue
    t = sub_df.t
    dpt = sub_df.mean_dpt
    
    dpt = dpt[t <= t_fit_max]
    t = t[t <= t_fit_max]
    
    dpt = dpt[t > t_fit_min]
    t = t[t > t_fit_min]
    
    if len(t) > 4:
        slope, intercept, r, p, se = linregress(minutes_per_time_point * t, dpt)
        slope_list_exp2_anterior.append(slope)
        
        t6hpi_index = np.where(t * minutes_per_time_point / 60 + start_hour > 6)[0][0]
        inten_6hpi_exp2_anterior.append(dpt.values[t6hpi_index])
        
        
# middle
this_df = df_exp2_middle
nuc_ids = this_df.particle.unique()
slope_list_exp2_posterior = []
inten_6hpi_exp2_posterior = []
for i in range(len(nuc_ids)):
    sub_df = this_df[this_df.particle == nuc_ids[i]]
    sub_df = sub_df.sort_values(by='t')
    if any(sub_df.groupby(by='t').size().values > 1):
        continue
    t = sub_df.t
    dpt = sub_df.mean_dpt
    
    dpt = dpt[t <= t_fit_max]
    t = t[t <= t_fit_max]
    
    dpt = dpt[t > t_fit_min]
    t = t[t > t_fit_min]
    
    if len(t) > 4:
        slope, intercept, r, p, se = linregress(minutes_per_time_point * t, dpt)
        slope_list_exp2_posterior.append(slope)
        
        t6hpi_index = np.where(t * minutes_per_time_point / 60 + start_hour > start_hour)[0][0]
        inten_6hpi_exp2_posterior.append(dpt.values[t6hpi_index])


In [177]:
median_anterior_exp1 = np.median(slope_list_anterior)
median_posterior_exp1 = np.median(slope_list_posterior)
median_anterior_exp2 = np.median(slope_list_exp2_anterior)
median_posterior_exp2 = np.median(slope_list_exp2_posterior)
plt.figure()
sig_x = 0.0
markersize = 18
"""anterior"""
x = 0

plt.plot(x + sig_x * np.random.normal(), median_anterior_exp1, 'ko', markersize=markersize, markerfacecolor=colors['e.coli'] + [0.5,], label='experiment 1',  markeredgewidth=2, markeredgecolor='k')
plt.plot(x + sig_x * np.random.normal(), median_anterior_exp2, 'kd', markersize=markersize, markerfacecolor=colors['e.coli'] + [0.5,], label='experiment 2',  markeredgewidth=2, markeredgecolor='k')



"""posterior"""
x = 1
plt.plot(x + sig_x * np.random.normal(), median_posterior_exp1, 'ko', markersize=markersize, markerfacecolor=colors['e.coli'] + [0.5,], label='_nolabel_',  markeredgewidth=2, markeredgecolor='k')
plt.plot(x + sig_x * np.random.normal(), median_posterior_exp2, 'kd', markersize=markersize, markerfacecolor=colors['e.coli'] + [0.5,], label='_nolabel_',  markeredgewidth=2, markeredgecolor='k')

plt.xticks([0, 1], labels=['anterior', 'middle'])
plt.xlim([-0.5, 1.5])
plt.ylabel('\nmedian diptericin-gfp \nactivation rate (a.u./min)', fontsize=fontsize)
plt.legend(fontsize=0.75 * fontsize, loc='upper left', bbox_to_anchor=(-0.5, 1.2), ncol=2, fancybox=True, facecolor=np.array([210, 180, 140]) / 255, framealpha=0.5)
ax = style_axes(plt.gca())


/home/brandon/anaconda3/envs/ome_zarr/lib/python3.9/site-packages/matplotlib/legend.py:469: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if facecolor == 'inherit':


In [178]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig2-SingleCellDynamics/median_rate_dot_plot.pdf')

## Plot scatter plot of rate vs intensity

In [170]:
plt.figure()
markersize = 12
markeredgewidth = 1
plt.plot(slope_list_anterior, inten_6hpi_exp1_anterior, 'ko', markerfacecolor=colors['e.coli'] + [0.25,],  markersize=markersize, markeredgewidth=markeredgewidth, markeredgecolor='k', label='experiment 1')
plt.plot(slope_list_posterior, inten_6hpi_exp1_posterior, 'ko', markerfacecolor=colors['e.coli'] + [0.25,],  markersize=markersize, markeredgewidth=markeredgewidth, markeredgecolor='k', label='_nolabel__')

plt.plot(slope_list_exp2_anterior, inten_6hpi_exp2_anterior, 'kd', markerfacecolor=colors['e.coli'] + [0.25,],  markersize=markersize, markeredgewidth=markeredgewidth, markeredgecolor='k', label='experiment 2')
plt.plot(slope_list_exp2_posterior, inten_6hpi_exp2_posterior, 'kd', markerfacecolor=colors['e.coli'] + [0.25,],  markersize=markersize, markeredgewidth=markeredgewidth, markeredgecolor='k', label='_nolabel_')

x = slope_list_anterior + slope_list_posterior + slope_list_exp2_anterior + slope_list_exp2_posterior
y = inten_6hpi_exp1_anterior + inten_6hpi_exp1_posterior + inten_6hpi_exp2_anterior + inten_6hpi_exp2_posterior

slope, intercept, r, p, se = linregress(x, y)
print(f'R^2 = {r ** 2}')
x = np.linspace(np.min(x), np.max(x), 5)
plt.plot(x, intercept + slope * np.array(x), 'k--', linewidth=4)

plt.xlabel('initial activation slope (a.u./min)', fontsize=fontsize)
plt.ylabel('diptericin-gfp \nfluorescence intensity \nat 6 hours post infection (a.u.)', fontsize=fontsize)
plt.legend(fontsize=0.75 * fontsize, loc='upper left', bbox_to_anchor=(0, 1.2), ncol=2, fancybox=True, facecolor=np.array([210, 180, 140]) / 255, framealpha=0.5)
ax = style_axes(plt.gca())


R^2 = 0.822074882774227


/home/brandon/anaconda3/envs/ome_zarr/lib/python3.9/site-packages/matplotlib/legend.py:469: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if facecolor == 'inherit':


In [171]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig2-SingleCellDynamics/rate_vs_inten_6hpi_scatter_plot.pdf')

## Plot distribution of all rates

In [265]:
def bgamma(x, alpha, beta, delta):
    x[x < 0] = 0
    Z = 2 + (alpha * delta / beta) * ((1 + alpha) * delta / beta - 2)
    out = (1 + (1 - delta * x) ** 2) * (beta ** alpha) * (x ** (alpha - 1)) * np.exp(-beta * x) / gamma(alpha) / Z
    out[out < 0] = 0
    
    return out

def negative_bgamma_llikelihood(p, x):
    alpha, beta, delta = p
    return -1 * np.sum(np.log(bgamma(x, alpha, beta, delta)))
    

In [293]:
markersize = 18
markeredgewidth = 2
plt.figure()
n_bins = 10 + 1
bins = np.linspace(0, 5, n_bins)
prob_dens_arr = np.zeros((2, n_bins - 1))

"""exp 1"""
all_slopes = np.array(slope_list_anterior + slope_list_posterior)# + slope_list_exp2_anterior + slope_list_exp2_posterior)
#all_slopes = np.array(slope_list_exp2_anterior + slope_list_exp2_posterior)
all_slopes = all_slopes[all_slopes > 0]
#n_bins = 10
#bins = np.linspace(np.min(all_slopes), np.max(all_slopes), n_bins)
#bins = np.linspace(0, 3.2, n_bins)
counts, _ = np.histogram(all_slopes, bins)
prob_dens = counts / np.sum(counts) / np.diff(bins)
prob_dens_arr[0] = prob_dens
mean_slope_1 = np.mean(all_slopes)

#bins = bins[:-1]

# p0 = (1.25, 0.5, 0.5)

# res = minimize(negative_bgamma_llikelihood, p0, args=all_slopes, bounds=((0, np.inf), (0, np.inf), (0, np.inf)))
# alpha, beta, delta = res.x
# plot_bins = np.linspace(np.min(bins), np.max(bins), 1000)

# #plt.plot(plot_bins, bgamma(plot_bins, alpha, beta, delta), 'k-', linewidth=4)
# plt.plot(bins, prob_dens, 'ko', markerfacecolor=colors['e.coli'],  markersize=markersize, markeredgewidth=markeredgewidth, markeredgecolor='k')




"""exp 2"""
#all_slopes = np.array(slope_list_anterior + slope_list_posterior + slope_list_exp2_anterior + slope_list_exp2_posterior)
all_slopes = np.array(slope_list_exp2_anterior + slope_list_exp2_posterior)
all_slopes = all_slopes[all_slopes > 0]
#n_bins = 10
#bins = np.linspace(np.min(all_slopes), np.max(all_slopes), n_bins)
#bins = np.linspace(0, 3.2, n_bins)
counts, _ = np.histogram(all_slopes, bins)
prob_dens = counts / np.sum(counts) / np.diff(bins)
prob_dens_arr[1] = prob_dens
mean_slope_2 = np.mean(all_slopes)

#bins = bins[:-1]

# p0 = (1.25, 0.5, 0.5)

# res = minimize(negative_bgamma_llikelihood, p0, args=all_slopes, bounds=((0, np.inf), (0, np.inf), (0, np.inf)))
# alpha, beta, delta = res.x
# plot_bins = np.linspace(np.min(bins), np.max(bins), 1000)

# #plt.plot(plot_bins, bgamma(plot_bins, alpha, beta, delta), 'k-', linewidth=4)
# plt.plot(bins, prob_dens, 'kd', markerfacecolor=colors['e.coli'],  markersize=markersize, markeredgewidth=markeredgewidth, markeredgecolor='k')

mean_prob_dens = np.mean(prob_dens_arr, axis=0)
std_prob_dens = np.std(prob_dens_arr, axis=0)

plt.errorbar(bins[:-1], mean_prob_dens, std_prob_dens)

mean_slope = (mean_slope_1 + mean_slope_2) / 2
plot_bins = np.linspace(0, 5, 1000)
exp_dist = mean_slope * np.exp(-plot_bins / mean_slope)
plt.plot(plot_bins, exp_dist, 'k-')


plt.xlabel('initial activation slope (a.u./min)', fontsize=fontsize)
plt.ylabel('probability density (min/a.u.)', fontsize=fontsize)
ax = style_axes(plt.gca())


In [296]:
mean_slope

1.3059596964374132

In [288]:
std_prob_dens

array([0.70300334, 0.09399333, 0.08731924, 0.08731924, 0.11716722,
       0.16091954, 0.06896552, 0.06896552, 0.01835373, 0.        ])

In [289]:
bins[:-1]

array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5])

In [341]:
X0 = np.exp(np.random.normal(loc=2, scale=1, size=100_000))
R = np.random.exponential(scale=900, size=100_000)

Xt = X0 + R

bins = np.logspace(0, 4, 100)
counts, _ = np.histogram(Xt, bins)
prob_dens = counts / np.sum(counts) / np.diff(bins)
plt.figure()
plt.plot(bins[:-1], prob_dens)
plt.xscale('log')
plt.yscale('log')

In [316]:
np.max(Xt)

1209.378224721931

In [318]:
np.min(X0)

0.18373883624527165

In [314]:
np.max(R)

373.48589106932803

In [324]:
R

2900.9254346984044